In [29]:
import os, json, tensorflow as tf
MODEL_FILENAME = os.environ.get("MODEL_FILENAME", "model/ResNet50V2_model.keras")

m = tf.keras.models.load_model(MODEL_FILENAME, compile=False)

In [30]:
print("== Basic ==")
print("Type:", type(m).__name__)
print("Trainable params:", m.count_params())

# shapes
try:
    print("Input shape:", m.input_shape)
except Exception:
    print("Input shape: (unknown, dynamic input)")

try:
    print("Output shape:", m.output_shape)
except Exception:
    print("Output shape: (unknown)")

# last layer / activation
try:
    last = m.layers[-1]
    act = getattr(last, "activation", None)
    act_name = getattr(act, "__name__", str(act))
    units = getattr(last, "units", None)
    print("Last layer:", last.name, f"(class={last.__class__.__name__})")
    print("Last activation:", act_name)
    print("Last units:", units)
except Exception as e:
    print("Last layer info error:", e)

# quick rule to infer task type
try:
    out_shape = m.outputs[0].shape  # tf.TensorShape
    n_classes = out_shape[-1] if out_shape.rank is not None else None
    if units == 1 and act_name in ("sigmoid", "linear"):
        print("Inference: Binary classification (sigmoid/threshold 0.5)")
    elif units == 2 and act_name == "softmax":
        print("Inference: 2-class softmax")
    elif isinstance(n_classes, int) and n_classes > 2 and act_name == "softmax":
        print(f"Inference: Multi-class softmax ({n_classes} classes)")
    else:
        print("Inference: Unknown/Custom (ดู activation/units ด้านบน)")
except Exception:
    pass

# dummy forward เพื่อตรวจ output จริง
try:
    # เดาว่าเป็นภาพ 224x224x3 ถ้าไม่ใช่ เปลี่ยนตาม input_shape
    H, W, C = 224, 224, 3
    if hasattr(m, "input_shape") and isinstance(m.input_shape, tuple) and len(m.input_shape) == 4:
        _, H, W, C = m.input_shape
    import numpy as np
    x = np.zeros((1, H, W, C), dtype="float32")
    y = m.predict(x, verbose=0)
    print("Predict OK. y.shape:", y.shape, "min/max:", float(y.min()), float(y.max()))
except Exception as e:
    print("Predict failed (shapeไม่ตรง?):", e)

== Basic ==
Type: Functional
Trainable params: 23566849
Input shape: (None, 224, 224, 3)
Output shape: (None, 1)
Last layer: dense_1 (class=Dense)
Last activation: sigmoid
Last units: 1
Predict OK. y.shape: (1, 1) min/max: 0.9900328516960144 0.9900328516960144
